In [1]:
using Pkg

Pkg.add("DataFrames")
Pkg.add("SQLite")
Pkg.add("LinearAlgebra")

using SQLite
using DataFrames
using LinearAlgebra

   Updating registry at `C:\Users\conno\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\conno\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\conno\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\conno\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\conno\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\conno\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\conno\.julia\environments\v1.5\Manifest.toml`


In [2]:
db = SQLite.DB(pwd()*"/names.db");

In [3]:
df = DBInterface.execute(db, "SELECT * FROM BabyNames") |> DataFrame;

In [4]:
gd = groupby(df, [:sex]);

In [5]:
Ny = nrow(unique(df, 1));

In [6]:
uniqueG = unique(gd[1],2);
Ng = nrow(uniqueG);

In [7]:
uniqueB = unique(gd[2],2);
Nb = nrow(uniqueB);

In [8]:
g_to_i = Dict{String, Integer}(string(uniqueG[i,2]) => i for i = 1:1:Ng);
i_to_g = Dict{Integer, String}(value => key for (key, value) in g_to_i);

In [9]:
b_to_i = Dict{String, Integer}(string(uniqueB[i,2]) => i for i = 1:1:Nb);
i_to_b = Dict{Integer, String}(value => key for (key, value) in b_to_i);

In [10]:
Fg = zeros(Float32, Ng, Ny);
Fb = zeros(Float32, Nb, Ny);

In [11]:
for i in eachrow(df)
    if(i[3] == "F")
        Fg[g_to_i[string(i[2])],i[1] - 1879] = i[4];
    else
        Fb[b_to_i[string(i[2])],i[1] - 1879] = i[4];
    end
end

In [12]:
Ty = zeros(Float32, 1, Ny);

In [13]:
for i = 1:1:Ny
    Ty[i] = sum(Fg[:,i]) + sum(Fb[:,i]);
end

In [14]:
Pg = zeros(Float32, Ng, Ny);
Pb = zeros(Float32, Nb, Ny);

In [15]:
for i = 1:1:Ny
    Pg[:,i] = Fg[:,i]/Ty[i];
    Pb[:,i] = Fb[:,i]/Ty[i];
end

In [16]:
Qg = zeros(Float32, Ng, Ny);
Qb = zeros(Float32, Nb, Ny);

In [17]:
for i = 1:1:Ng
    Qg[i,:] = normalize(Pg[i,:]);
end
for i = 1:1:Nb
    Qb[i,:] = normalize(Pb[i,:]);
end

In [18]:
#x = Qg*transpose(Qb)
#y = findmax(x)

In [19]:
#global girl = i_to_g[y[2][1]]
#global boy = i_to_b[y[2][2]] 
#print("Boy and girl pair: (", boy, ", " , girl,")", " Value: ", y[1])

In [20]:
global viewg = [];
global viewb = [];

for i = 1:6833:61498
    if(i == 61498)
        push!(viewg, @view Qg[i:i+6834,:]);

    else
        push!(viewg, @view Qg[i:i+6832,:]);
    end
end

for i = 1:4205:37846
    if(i == 37846)
        push!(viewb, @view Qb[i:i+4208,:]);
    else
        push!(viewb, @view Qb[i:i+4204,:]);
    end
end

#= 
Testing for boundries
        print(i,":", i+6834, "\n")
        print(size(@view Qg[i:i+6834,:]),"\n")

        print(i,":", i+6832, "\n")
        print(size(@view Qg[i:i+6832,:]),"\n")
        --------------------------------------
        print(i,":", i+4208, "\n")
        print(size(@view Qb[i:i+4208,:]),"\n")

        print(i,":", i+4204, "\n")
        print(size(@view Qb[i:i+4204,:]),"\n")
=#

In [21]:
global max_cosine = 0;
global girl = "";
global boy = "";
#l = ReentrantLock()
#global hold = [];

for i = 1:10
Threads.@threads for j = 1:10
        temp_transpose = viewg[i]*transpose(viewb[j])
        found = findmax(temp_transpose)
        
        # To construct matrix from the 100 parts
        #=
        #lock is needed to make sure hold isn't written at the same time
        lock(l)
        try
            push!(hold, temp_transpose)
        finally
            unlock(l)
        end
        =#
        
        if(max_cosine < found[1])
            global max_cosine = found[1]
            if(i == 10)
                global girl = i_to_g[found[2][1]+(6833*(i-1))+2]
                global boy = i_to_b[found[2][2]+(4205*(j-1))+4]   
            else
                global girl = i_to_g[found[2][1]+6833*(i-1)]
                global boy = i_to_b[found[2][2]+4205*(j-1)]  
            end
        end
    end
end

In [22]:
print("Boy and girl pair: (", boy, ", " , girl,")")

Boy and girl pair: (Alwood, Amilda)

In [23]:
Threads.nthreads()

8